In [3]:
import importlib, os, numpy as np
import ifimage_tools                           # 你自己的 utils 包
importlib.reload(ifimage_tools)                # 热重载
from tqdm.auto import tqdm                     # 可选：进度条
import os
import numpy as np
from stardist.matching import matching

In [4]:
image_dir  = "Reorgnized Ground Truth"
masks_dir  = "merged_mask"
dataset = ifimage_tools.IfImageDataset(image_dir, masks_dir, {})
dataset.load_data()

for sid in ["6390", "8408", "8406", "8405v2", "8405", "8407"]:
    dataset.samples.pop(sid, None)

# --------------------------- 2. 分割 & 保存 -----------------------------------
output_root = "cell_masks"            # 新目录，不要跟 nuclei 混
os.makedirs(output_root, exist_ok=True)

In [5]:
for sid, sample in dataset.samples.items():
    if sample.marker is None: continue
    m = sample.marker
    print(f"{sid}: marker ndim={m.ndim}, shape={m.shape}, min={m.min():.3f}, max={m.max():.3f}")
    break

5803: marker ndim=2, shape=(1040, 1388), min=10.000, max=255.000


In [6]:
dataset.samples.items()

dict_items([('5803', <ifimage_tools.ImageSample object at 0x7f5c69d7b370>), ('12781', <ifimage_tools.ImageSample object at 0x7f5c69d3a8c0>), ('12779', <ifimage_tools.ImageSample object at 0x7f5c69d7a890>), ('12786', <ifimage_tools.ImageSample object at 0x7f5c69dc8790>), ('10594', <ifimage_tools.ImageSample object at 0x7f5c69d92bc0>), ('8408jan22', <ifimage_tools.ImageSample object at 0x7f5c69d93e80>), ('15591', <ifimage_tools.ImageSample object at 0x7f5c69d78100>), ('12793', <ifimage_tools.ImageSample object at 0x7f5ede641780>), ('15972', <ifimage_tools.ImageSample object at 0x7f5c69d7a770>), ('12794', <ifimage_tools.ImageSample object at 0x7f5c6a2b3490>), ('1120', <ifimage_tools.ImageSample object at 0x7f5ede641750>), ('12795', <ifimage_tools.ImageSample object at 0x7f5ede640e50>), ('12792', <ifimage_tools.ImageSample object at 0x7f5ede640fa0>), ('12787', <ifimage_tools.ImageSample object at 0x7f5ede640ca0>), ('12780', <ifimage_tools.ImageSample object at 0x7f5c6a2b36d0>), ('2529', <i

In [ ]:
methods = ["cellpose","cellpose2chan"]

for sample_id, sample in tqdm(dataset.samples.items(), desc="cell seg"):
    # 跑细胞质阳性分割
    sample.get_positive_cyto_pipline(methods=methods)

    # mkdir cell_masks/<sample_id>/
    sample_dir = os.path.join(output_root, sample_id)
    os.makedirs(sample_dir, exist_ok=True)

    # 保存每种算法的 mask
    for m in methods:
        mask = sample.cyto_positive_masks.get(m)
        if mask is None:
            print("失败！")
            continue                      # 方法失败/未跑
        out_path = os.path.join(sample_dir, f"{m}.npy")
        np.save(out_path, mask.astype(np.uint32))
        print(f"[{sample_id}] saved {m} → {out_path}")


cell seg:   1%|          | 1/100 [00:00<00:43,  2.26it/s]

[5803] saved watershed_only_cyto → cell_masks/5803/watershed_only_cyto.npy


cell seg:   2%|▏         | 2/100 [00:00<00:41,  2.33it/s]

[12781] saved watershed_only_cyto → cell_masks/12781/watershed_only_cyto.npy


cell seg:   3%|▎         | 3/100 [00:01<00:43,  2.25it/s]

...

[4683] saved watershed_only_cyto → cell_masks/4683/watershed_only_cyto.npy


cell seg:  99%|█████████▉| 99/100 [00:44<00:00,  2.24it/s]

[4319] saved watershed_only_cyto → cell_masks/4319/watershed_only_cyto.npy


cell seg: 100%|██████████| 100/100 [00:45<00:00,  2.21it/s]

[1109] saved watershed_only_cyto → cell_masks/1109/watershed_only_cyto.npy
